In [1]:
import torch
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import torch.nn as nn  
import torch.optim as optim
import math
from torchsummary import summary

In [2]:
import os
import pandas as pd
import traceback

In [60]:
def dataloaderV2(dossier, etat):
    # on part d'un dossier racine où chacun des dossiers contenant les données est
    # initialise data = tableau final
    data = []
    # initialise le di;ctionnaire qui va stocker toutes les valeurs de session et class par individu
    
    #on va parcourir tous les dossier du premier dossier
    for dos in os.listdir(dossier):
        #on met à jour le path pour pouvoir parcourir le sous dossier
        dos_path = os.path.join(dossier, dos)
        
        sorted_files = sorted(os.listdir(dos_path))
        
        #on vérifie que c'est bien un dossier
        if os.path.isdir(dos_path):
            # on va parcourir les fichiers contenant les datas
            n_va = 0
            for fichier in sorted_files:
                #on initialise l'indice de VA qui va vous permettre de parcourir le data final par colonnes
                #vérifie que c'est bien un fichier csv
                if fichier.endswith(".csv") and fichier.split("_")[0].lower()== etat:
                    #print("fichier", fichier) #okay va dans tous les fichiers
                    #met à jour le path pour pouvoir ouvrir le fichier
                    filepath = os.path.join(dos_path, fichier)
                    try:
                        df = pd.read_csv(filepath, header = 0, low_memory=False)
                    except Exception as e:
                        print(f"Error reading file '{va}': {e}")
                        
                    for indiv in range(len(data)):
                        # Expand this individual's list if it doesn't have enough slots
                        while len(data[indiv]) <= n_va:
                            required_length = n_va + 1
                            current_length = len(data[indiv])
                            additional_slots_needed = required_length - current_length
                            if additional_slots_needed > 0:
                                data[indiv].extend([None] * additional_slots_needed)
                            
                    # on va otérer sur toutes les datas de ce fichier, 
                    # avec num indiv indice de 0 ua bout, index = index de lignes et row la data
                    for indiv, (index, row) in enumerate(df.iterrows()):
                        try:
                            #print(f"before indiv: {indiv}, n_va: {n_va}, len(data[indiv]): {len(data[indiv])}")
                            # Ensure the sublist exists and is of the correct length
                            while len(data) <= indiv:
                                #print("data avant ajustement pas d'indiv : ", data) okay les datas sont correctes
                                # Extend data list with empty lists as needed
                                #data.extend([[] for _ in range(indiv - len(data) + 1)])
                                data.extend([[]] * (indiv - len(data) + 1))
                                # Diagnostic prints to understand the structure just before the error occurs
                                #print(f"Attempting assignment to data[{indiv}][{n_va}].")
                                #print(f"len(data): {len(data)}, len(data[{indiv}]): {len(data[indiv])}.")

                                #print("data avant ajustement pas d'indiv : ", data)okay les datas sont correctes
                            while len(data[indiv]) <= n_va:
                                #print("rien dans les indiv = pas de va AVANT : ", data) okay les datas sont correctes
                                # Extend the sublist with None values as placeholders
                                #data[indiv].extend([None for _ in range(n_va - len(data[indiv]) + 1)])
                                data[indiv].append([])    
                                #data[indiv].append(None)
                            #print(f"after indiv: {indiv}, n_va: {n_va}, len(data[indiv]): {len(data[indiv])}")
                            
                                #print("rien dans les indiv = pas de va APRES : ", data)
                            # Now safe to assign the value
                            #print(f"Right before assignment to data[{indiv}][{n_va}]: len(data): {len(data)}, len(data[{indiv}]): {len(data[indiv])}, len(data[{indiv}][{n_va}]): 'Will be set'")

                           
                            # Extract session and class if already present
                            if len(data[indiv]) >= 2:
                                #print("session class deja la AVANT : ", data[indiv])# okay les datas sont correctes
                                session_data, class_data = data[indiv][-2:]  # Extract existing session and class
                                data[indiv] = data[indiv][:-2]  # Remove session and class from the current list
                                #print("session class deja la APRES : ", data) okay les datas sont correctes
                                #print("session class deja la APRES : ", data[indiv]) okay les datas sont correctes
                                #print("session : ", session_data, " class : ", class_data) okay les datas sont correctes
                            else:
                                #print("recup session class depuis datas")
                                session_data, class_data = row.iloc[-2], row.iloc[-1]  # Get new session and class
                                #print("session : ", session_data, " class : ", class_data) print("session : ", session_data, " class : ", class_data

                            # For each individual, append the new row data, excluding session and class
                            #row_data = row.iloc[:-2].tolist()
                            #data[indiv].extend(row_data)  # Append the new row data
                            
                           # print("data avant append nouvelle va : ", data[indiv][n_va]) okay les datas sont correctes
                            data[indiv].append(row.iloc[:-2].tolist())
                            #print("data apres append nouvelle va : ", data[indiv][n_va]) okay les datas sont correctes
                            # Re-append the session and class data at the end
                            #print("data avant append nouvelle va append session/class: ", data[indiv]) okay les datas sont correctes
                            data[indiv].append([session_data])
                            #print("data apres append nouvelle va append session: ", data[indiv]) okay les datas sont correctes
                            data[indiv].append([class_data])
                            #print("data apres append nouvelle va append class: ", data[indiv]) okay les datas sont correctes
                            
                        #except (IndexError, TypeError):
                        except (IndexError, TypeError) as e:
                            print(f"error: {type(e).name}")
                            #if isinstance(row.iloc[:-2], pd.Series):
                             #   data.append([row.iloc[:-2].tolist()])
                            #traceback.print_exc()
                            # Properly append a new sublist if catching an IndexError before this logic
                            #if isinstance(row.iloc[:-2], pd.Series):
                            #    data.append([row.iloc[:-2].tolist()])
                            #    print("error")
                            #else:
                                # Handle unexpected data format
                             #   data.append([None])
                            
                    n_va +=1

    return data 

def nettoyage(dat):
    for i in range(len(dat)):
        dat[i].pop(0)
        dat[i].pop(-2)
        dat[i].pop(-1)
    return dat

directory_path = "./RAW"
data_n = dataloaderV2(directory_path, "normal")
data_f = dataloaderV2(directory_path, "fatigue")

data_n = nettoyage(data_n)
data_f = nettoyage(data_f)

data_n0 = []
data_n1 = []
split_index = len(data_n[0]) // 2
for i in range(len(data_n)):
    data_n0.append(data_n[i][:split_index])
    data_n1.append(data_n[i][split_index:])
    
data_n = data_n0+data_n1

data_f0 = []
data_f1 = []
split_index = len(data_f[0]) // 2
for i in range(len(data_f)):
    data_f0.append(data_f[i][:split_index])
    data_f1.append(data_f[i][split_index:])
    
data_f = data_f0+data_f1

for i in range(len(data_n)):
    data_n[i].append([0])
for i in range(len(data_f)):
    data_f[i].append([1])
    
data = data_f + data_n
#print("LEN DATA")
#print(len(data))
#print("LEN DATA[0]")
#print(len(data[0]))
#print("LEN COntainer")
#for i in range(len(data[0])):
#    print(len(data[0][i]))
#print(data[2000][-1])

In [61]:
DataLen = len(data)
ModelNum = len(data[0])
inputs = [0] * ModelNum  # Create a list of size n with all elements initialized to 0
for i in range(ModelNum):
    inputs[i] = len(data[0][i])
inputs_cuda = [torch.randn(1, inp).to('cuda') for inp in inputs]

In [71]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = pd.DataFrame(data)
        self.data = self.data.apply(pd.to_numeric, errors='coerce').fillna(0) 
        self.features = self.data.iloc[:, :-1].values
        self.targets = self.data.iloc[:, -1].values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = torch.tensor(self.features[idx], dtype=torch.float32)
        target = torch.tensor(self.targets[idx], dtype=torch.float32)  
        return features, target
    
    def collate_fn(self, batch):
        features, targets = zip(*batch)
        return torch.stack(features), torch.stack(targets)

dataset = CustomDataset(data)

train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)

train_loader = DataLoader(train_set, batch_size=1, shuffle=True, collate_fn=dataset.collate_fn)
test_loader = DataLoader(test_set, batch_size=1, shuffle=True, collate_fn=dataset.collate_fn)

In [73]:
class MLP(nn.Module):
    def __init__(self,input):
        super().__init__()
        self.in_dim = input
        self.fc = []
        layer = 2**int(math.log2(self.in_dim))
        self.fc.append(nn.Linear(self.in_dim, layer))
        layer = int(layer/2)
        for i in range(int(math.log2(self.in_dim))):
            self.fc.append(nn.Linear(layer*2, layer))
            layer = int(layer/2)
        self.relu = nn.LeakyReLU(negative_slope=0.001)
        self.log_softmax = nn.LogSoftmax(dim=1)
        #print("init a nn with an input of ",input," layer of " , 2**int(math.log2(self.in_dim)))
        #for i in range(int(math.log2(self.in_dim)+1)):
        #    print(self.fc[i])
    
    def forward(self, x):
        a = []
        a.append(self.relu(self.fc[0](x)))
        for i in range(len(self.fc)-1):
           a.append(self.relu(self.fc[i+1](a[i])))
        output = self.log_softmax(a[len(a)-1])
        return output    
    
model = []
for i in range(ModelNum):
    model.append(MLP(inputs[i]))
FinalModel = MLP(ModelNum-1)

#for i in range(ModelNum):
#    #summary(model[i], (5,))
#    if any(p.is_cuda for p in model[i].parameters()):
#        print("Model is on CUDA")
#    else:
#        print("Model is on CPU")

# Summarize models

In [74]:
class SuperNode(nn.Module):
    def __init__(self,Model,FinalNode):
        super(SuperNode, self).__init__()
        self.Model = nn.ModuleList(Model) 
        self.finalNode = FinalNode
        #print("init a nn with an input of ",input," layer of " , 2**int(math.log2(self.in_dim)))
        #for i in range(int(math.log2(self.in_dim)+1)):
        #    print(self.fc[i])
        self.fc = nn.Linear(ModelNum-1, 1)
    
    def forward(self, x):
        def forward(self, x):
            tab = []
            for i in range(len(self.Model)):
                tab.append(self.Model[i](x[i]))
            output = self.finalNode(tab)
            return output

SuperNN = SuperNode(model,FinalModel)

In [75]:
criterion = nn.MSELoss()  # Utilisation de MSELoss pour la régression
optimizer = optim.SGD(SuperNN.parameters(), lr=0.0001)

for epoch in range(5):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        
        # Debugging print statements
        print("Inputs shape:", inputs)
        print("Labels shape:", labels)
        
        
        optimizer.zero_grad()
        outputs = SuperNN(inputs)
        loss = criterion(outputs, labels)  # Squeeze pour éliminer les dimensions inutiles
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if (i + 1) % 2000 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

Inputs shape: tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])
Labels shape: tensor([0.])


AttributeError: 'NoneType' object has no attribute 'size'